In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from bin import *

from IPython.display import clear_output
%load_ext autoreload
%autoreload 2

In [2]:
mpl.use("pgf")

mpl.rcParams.update({
    "pgf.texsystem": "lualatex",
    'font.family': 'serif',
    'font.size' : 8,
    'text.usetex': True,
    'pgf.rcfonts': False,
    'pgf.preamble': tex_preamble,
})

In [3]:
n = 200
Λ = np.hstack([np.linspace(-6,-1,n//2),np.linspace(1,10,n//2)])

A = np.diag(Λ)

b = np.ones(n)
b /= np.linalg.norm(b)

f = lambda x: 1/x

fAb = np.diag(f(Λ))@b

In [4]:
k_max = 51
Q_ro,(α_ro,β_ro) = lanczos_reorth(A,b,k_max,reorth=k_max)

T = np.zeros((k_max+1,k_max))
T[:k_max,:k_max] = np.diag(α_ro) + np.diag(β_ro[:-1],1) + np.diag(β_ro[:-1],-1)
T[k_max,k_max-1] = β_ro[-1]

res_CG = np.full(k_max,np.nan)
res_MR = np.full(k_max,np.nan)
for k in range(k_max):
    res_CG[k] = np.linalg.norm(b - A@lanczos_FA(f,Q_ro,α_ro,β_ro,k))

    Tk1k = T[:k+1,:k]
    Qk = Q_ro[:,:k]
    
    e1 = np.zeros(k+1)
    e1[0] = 1
    sol = np.linalg.lstsq(Tk1k,e1,rcond=None)
    x_MR = np.linalg.norm(b)*Qk@sol[0]
    res_MR[k] = np.linalg.norm(b)*sol[1][0]**.5


In [5]:
fig = plt.figure(figsize=(figure_width*cm,figure_height*cm))

ax = fig.add_axes([left, bottom, width, height])

ax.plot(res_CG,**line_styles['l1'],label='CG')
ax.plot(res_MR,**line_styles['l2'],label='MINRES')
#ax.plot(np.sqrt(np.arange(k_max)+1)*res_MR)

ax.grid(True,linestyle=':',linewidth=.5)

ax.set_xlabel('number of matvecs: $k$')
ax.set_ylabel('residual norm')
ax.set_yscale('log')

plt.savefig('imgs/CG_MINRES.pdf')
plt.close()